# Explore here

In [ ]:
import sys
sys.path.append("/Users/jesus/.pyenv/versions/3.10.16/lib/python3.10/site-packages")
import pandas as pd
print(pd.__version__)
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import set_random_seed
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense
set_random_seed(42)
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
total_data = pd.read_csv("/Users/jesus/Desktop/4geeks/4geeks - INTRO TO DEEP LEARNING/jesus-cuenca-machine-learning-python-template/data/raw/sampleSubmission.csv")

total_data.head()

In [ ]:
total_data.info

In [ ]:
import zipfile
import os
import shutil

# Rutas para los archivos zip y las rutas de extracción
train_zip_path = "/Users/jesus/Desktop/4geeks/4geeks - INTRO TO DEEP LEARNING/jesus-cuenca-machine-learning-python-template/data/processed/train.zip"
train_extract_path = "/Users/jesus/Desktop/4geeks/4geeks - INTRO TO DEEP LEARNING/jesus-cuenca-machine-learning-python-template/data/processed/train/train"
cat_folder = "/Users/jesus/Desktop/4geeks/4geeks - INTRO TO DEEP LEARNING/jesus-cuenca-machine-learning-python-template/data/processed/train/cat"
dog_folder = "/Users/jesus/Desktop/4geeks/4geeks - INTRO TO DEEP LEARNING/jesus-cuenca-machine-learning-python-template/data/processed/train/dog"

# Crear carpetas de cat y dog si no existen
os.makedirs(cat_folder, exist_ok=True)
os.makedirs(dog_folder, exist_ok=True)

# Extraer archivos si no están ya descomprimidos
if not os.path.exists(train_extract_path):
    with zipfile.ZipFile(train_zip_path, 'r') as zip_ref:
        zip_ref.extractall(os.path.dirname(train_extract_path))

print("✅ Archivos descomprimidos correctamente.")

# Mostrar el contenido de la carpeta de extracción
print("Contenido de la carpeta de extracción:")
print(os.listdir(os.path.dirname(train_extract_path)))

# Mover imágenes a las carpetas correspondientes
for img_file in os.listdir(train_extract_path):
    if img_file.startswith("cat") and (img_file.endswith('.jpg') or img_file.endswith('.jpeg') or img_file.endswith('.png')):
        shutil.move(os.path.join(train_extract_path, img_file), os.path.join(cat_folder, img_file))
        print(f"Moviendo {img_file} a la carpeta cat.")
    elif img_file.startswith("dog") and (img_file.endswith('.jpg') or img_file.endswith('.jpeg') or img_file.endswith('.png')):
        shutil.move(os.path.join(train_extract_path, img_file), os.path.join(dog_folder, img_file))
        print(f"Moviendo {img_file} a la carpeta dog.")

# Vaciar la carpeta train (eliminar solo archivos)
for img_file in os.listdir(train_extract_path):
    file_path = os.path.join(train_extract_path, img_file)
    if os.path.isfile(file_path):  # Asegúrate de que sea un archivo
        os.remove(file_path)

print("✅ Carpeta train vaciada.")

# Eliminar la carpeta train
shutil.rmtree(train_extract_path)
print("✅ Carpeta 'train' eliminada.")

print(f"Número de imágenes en cat: {len(os.listdir(cat_folder))}")
print(f"Número de imágenes en dog: {len(os.listdir(dog_folder))}")



In [ ]:
import matplotlib.pyplot as plt
import os
import cv2

# Función para cargar y mostrar imágenes
def mostrar_imagenes(carpeta, titulo, num_imagenes=9):
    # Obtener una lista de todas las imágenes en la carpeta
    imagenes = os.listdir(carpeta)
    
    # Seleccionar solo las primeras 'num_imagenes' imágenes
    imagenes = imagenes[:num_imagenes]

    # Crear una figura para mostrar las imágenes
    fig, axes = plt.subplots(3, 3, figsize=(10, 10))
    axes = axes.flatten()  # Aplanar la matriz de ejes

    for ax, img_file in zip(axes, imagenes):
        img_path = os.path.join(carpeta, img_file)
        # Cargar la imagen
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convertir de BGR a RGB
        ax.imshow(img)
        ax.set_title(f"{titulo} {img_file}")
        ax.axis('off')  # Ocultar los ejes

    plt.tight_layout()
    plt.show()

# Mostrar 9 imágenes de gatos
mostrar_imagenes(cat_folder, "Gato")

# Mostrar 9 imágenes de perros
mostrar_imagenes(dog_folder, "Perro")


In [ ]:
# Generador de datos para entrenamiento y validación
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalización de píxeles (0-1)
    validation_split=0.2  # 80% entrenamiento, 20% validación
)

# Generador para datos de entrenamiento
train_generator = train_datagen.flow_from_directory(
    "/Users/jesus/Desktop/4geeks/4geeks - INTRO TO DEEP LEARNING/jesus-cuenca-machine-learning-python-template/data/processed/train",
    target_size=(224, 224),  # Asegúrate de que coincida con el tamaño de entrada del modelo
    batch_size=32,
    class_mode="binary",  # Cambiado a "binary"
    subset="training"
)

val_generator = train_datagen.flow_from_directory(
    "/Users/jesus/Desktop/4geeks/4geeks - INTRO TO DEEP LEARNING/jesus-cuenca-machine-learning-python-template/data/processed/train",
    target_size=(224, 224),  # Asegúrate de que coincida con el tamaño de entrada del modelo
    batch_size=32,
    class_mode="binary",  # Cambiado a "binary"
    subset="validation"
)

# Generador para datos de prueba (puedes cambiar esto según sea necesario)
test_datagen = ImageDataGenerator(rescale=1./255)  # Solo normalización
test_generator = test_datagen.flow_from_directory(
    "/Users/jesus/Desktop/4geeks/4geeks - INTRO TO DEEP LEARNING/jesus-cuenca-machine-learning-python-template/data/processed/test1",
    target_size=(224, 224),  # Asegúrate de que coincida con el tamaño de entrada del modelo
    batch_size=32,
    class_mode=None,  # Sin etiquetas para el conjunto de prueba
    shuffle=False
)

print("Datos cargados correctamente.")


In [ ]:
model = Sequential()
model.add(Conv2D(input_shape = (224,224,3), filters = 64, kernel_size = (3,3), padding = "same", activation = "relu"))
model.add(Conv2D(filters = 64,kernel_size = (3,3),padding = "same", activation = "relu"))
model.add(MaxPool2D(pool_size = (2,2),strides = (2,2)))
model.add(Conv2D(filters = 128, kernel_size = (3,3), padding = "same", activation = "relu"))
model.add(Conv2D(filters = 128, kernel_size = (3,3), padding = "same", activation = "relu"))
model.add(MaxPool2D(pool_size = (2,2),strides = (2,2)))
model.add(Conv2D(filters = 256, kernel_size = (3,3), padding = "same", activation = "relu"))
model.add(Conv2D(filters = 256, kernel_size = (3,3), padding = "same", activation = "relu"))
model.add(Conv2D(filters = 256, kernel_size = (3,3), padding = "same", activation = "relu"))
model.add(MaxPool2D(pool_size = (2,2),strides = (2,2)))
model.add(Conv2D(filters = 512, kernel_size = (3,3), padding = "same", activation = "relu"))
model.add(Conv2D(filters = 512, kernel_size = (3,3), padding = "same", activation = "relu"))
model.add(Conv2D(filters = 512, kernel_size = (3,3), padding = "same", activation = "relu"))
model.add(MaxPool2D(pool_size = (2,2),strides = (2,2)))
model.add(Conv2D(filters = 512, kernel_size = (3,3), padding = "same", activation = "relu"))
model.add(Conv2D(filters = 512, kernel_size = (3,3), padding = "same", activation = "relu"))
model.add(Conv2D(filters = 512, kernel_size = (3,3), padding = "same", activation = "relu"))
model.add(MaxPool2D(pool_size = (2,2),strides = (2,2)))
model.add(Flatten())
model.add(Dense(units = 4096,activation = "relu"))
model.add(Dense(units = 4096,activation = "relu"))
model.add(Dense(units = 2, activation = "softmax"))

In [ ]:
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",  
    metrics=["accuracy"]
)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

callbacks = [
    ModelCheckpoint(filepath='best_model.h5', save_best_only=True, monitor='val_accuracy', mode='max'),
    EarlyStopping(monitor='val_loss', patience=5, mode='min')
]

# Entrenar el modelo
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=1,  # Cambia el número de épocas según sea necesario
    callbacks=callbacks
)

In [ ]:
# Evaluar el modelo en el conjunto de entrenamiento
train_loss, train_accuracy = model.evaluate(train_generator)

print(f"Accuracy en el conjunto de entrenamiento: {train_accuracy}")

# Evaluar el modelo en el conjunto de validación
val_loss, val_accuracy = model.evaluate(val_generator)

print(f"Accuracy en el conjunto de validación: {val_accuracy}")

In [ ]:
# Evaluar el modelo en el conjunto de prueba
test_loss, test_acc = model.evaluate(test_generator, verbose=2)

print('\nTest accuracy:', test_acc)

In [ ]:
# Guardar el modelo en formato Keras
model.save("keras_28x28-128-10_42.keras")